In [1]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
med=pd.read_csv('medical/medical_data.csv')


In [3]:
train_data, test_data = train_test_split(med, test_size=0.3)

train_symptoms=train_data.drop(columns=['diagnosis'])
train_labels=train_data['diagnosis']               
    
test_symptoms=test_data.drop(columns=['diagnosis'])
test_labels=test_data['diagnosis']               
    


In [4]:
DT = DecisionTreeClassifier()
KNN = KNeighborsClassifier()
MNB = MultinomialNB()


In [5]:

#=======================================================================================
#build the parameter grid
KNN_grid = [{'n_neighbors': [1,3,5,7,9,11,13,15,17], 'weights':['uniform','distance']}]

#build a grid search to find the best parameters
gridsearchKNN = GridSearchCV(KNN, KNN_grid, cv=5)

#run the grid search
gridsearchKNN.fit(train_symptoms,train_labels)

#=======================================================================================

#best parameters
print(gridsearchKNN.best_params_)

#predictions using the best parameters
KNN_pred=gridsearchKNN.predict(test_symptoms)
print(accuracy_score(KNN_pred,test_labels))


{'n_neighbors': 11, 'weights': 'uniform'}
0.8166666666666667


In [6]:

#build the parameter grid
DT_grid = [{'max_depth': [3,4,5,6,7,8,9,10,11,12],'criterion':['gini','entropy']}]

#build a grid search to find the best parameters
gridsearchDT  = GridSearchCV(DT, DT_grid, cv=5)

#run the grid search
gridsearchDT.fit(train_symptoms,train_labels)

print(gridsearchDT.best_params_)

#predictions using the best parameters
DT_pred=gridsearchDT.predict(test_symptoms)
print(accuracy_score(DT_pred,test_labels))

{'criterion': 'gini', 'max_depth': 6}
0.82


In [7]:

#=======================================================================================

#build the parameter grid
MNB_grid = [ {'alpha':[0.5,1,1.5,2],'fit_prior':[True,False]}]

#build a grid search to find the best parameters
gridsearchMNB  = GridSearchCV(MNB, MNB_grid, cv=5)

#run the grid search
gridsearchMNB.fit(train_symptoms,train_labels)

#=======================================================================================

print(gridsearchMNB.best_params_)

#predictions using the best parameters
MNB_pred=gridsearchMNB.predict(test_symptoms)
print(accuracy_score(MNB_pred,test_labels))

{'alpha': 2, 'fit_prior': True}
0.8283333333333334


In [8]:
#combining Voting with Gridsearch
predictors=[('dt', DT),('knn', KNN),('mnb', MNB)]

# Create a joint grid for all 3 models. Use the name of the model followed by a __ as a prefix for each parameter name
voting_grid = {
    'knn__n_neighbors': [1,3,5,7,9], 
    'knn__weights':['uniform','distance'],
    'mnb__alpha':[0.5,1,1.5,2],
    'mnb__fit_prior':[True,False],
    'dt__max_depth': [3,4,5,6,7],
    'dt__criterion':['gini','entropy']}
    
VT=VotingClassifier(predictors)

VT_tuned = GridSearchCV(VT, voting_grid, cv=5)

VT_tuned.fit(train_symptoms,train_labels)

#use the VT classifier to predict
predicted=VT_tuned.predict(test_symptoms)

#print the accuracy
print (accuracy_score(predicted,test_labels))




0.8266666666666667
